In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Input Target Tweet --> Labels ---> Label Descriptions

In [2]:
import os
import warnings
import pandas as pd
import torch
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

[2025-05-24 01:20:21,921] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/sohampoddar/anaconda3/envs/llm/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/sohampoddar/anaconda3/envs/llm/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/sohampoddar/anaconda3/envs/llm/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/sohampoddar/anaconda3/envs/llm/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda-12/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/sohampoddar/anaconda3/envs/llm/compiler_compat/ld: /usr/local/cuda-12/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/sohampoddar/anaconda3/envs/llm/compiler_compat/ld: /usr/local/cuda-12/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/sohampodda

In [3]:
class TrainDataset(Dataset):
    def __init__(self,csv_file,root_dir,transform=None):
        self.train_data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.sentences = self.train_data['input']
        self.text_labels = self.train_data['target']

    def __len__(self):
        return len(self.train_data)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        target = self.text_labels[idx]
        print(sentence)
        print(target)
        sample = {'sentence': sentence, 'text_label': target}
        
        return sample

In [4]:
class TestDataset(Dataset):
    def __init__(self,csv_file,root_dir,transform=None):
        self.test_data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.sentences = self.test_data['input']
        self.text_labels= self.test_data['target']

    def __len__(self):
        return len(self.test_data)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        target = self.text_labels[idx]
        print(sentence)
        print(target)
        sample = {'sentence': sentence, 'text_label': target}

        return sample

In [5]:
caves_train = TrainDataset(csv_file='/home/sohampoddar/HDD2/utsav/Dataset/train_caves.csv',root_dir='/home/sohampoddar/HDD2/utsav/Dataset')
caves_test = TestDataset(csv_file='/home/sohampoddar/HDD2/utsav/Dataset/test_caves_new.csv',root_dir='/home/sohampoddar/HDD2/utsav/Dataset/')
# print(caves_train[0])

In [6]:
dataset_train = Dataset.from_dict(
    {
        "sentence": list(caves_train.sentences),
        "text_label": list(caves_train.text_labels),
    }
)
dataset_test = Dataset.from_dict(
    {
        "sentence": list(caves_test.sentences),
        "text_label": list(caves_test.text_labels),
    }
)

In [7]:
# tokenizer = AutoTokenizer.from_pretrained("t5-base", cache_dir="/home/sohampoddar/HDD2/utsav/transformers_cache")
tokenizer = AutoTokenizer.from_pretrained("t5-large", cache_dir="/home/sohampoddar/HDD2/utsav/transformers_cache") 

In [8]:
max_length = 128
batch_size = 8

In [9]:
def preprocess_function(examples):
    inputs = examples['sentence']
    targets = examples['text_label']
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=30, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs


processed_datasets_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=['sentence', 'text_label'],
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

processed_datasets_test = dataset_test.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=['sentence', 'text_label'],
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/6957 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/987 [00:00<?, ? examples/s]

In [10]:
train_dataset = processed_datasets_train
eval_dataset = processed_datasets_test

# model_name_or_path = "t5-base"
model_name_or_path = "t5-large"
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)

eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

lora_config = LoraConfig(r=8, lora_alpha=32, target_modules=["q", "v"], lora_dropout=0.05, bias="none",
                         task_type=TaskType.SEQ_2_SEQ_LM)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, cache_dir='./', device_map='auto')
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

output_dir = "lora-flan-t5-large-train"

trainable params: 2,359,296 || all params: 740,027,392 || trainable%: 0.3188


In [11]:
device = "cuda"
# model_name_or_path = "google/flan-t5-base"
model_name_or_path = "google/flan-t5-large"
# tokenizer_name_or_path = "google/flan-t5-base"
tokenizer_name_or_path = "google/flan-t5-large"
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=5e-4,  # higher learning rate
    num_train_epochs=4,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

# train model
trainer.train()
# peft_model_id = "lora_flan_t5_base_train"
peft_model_id = "lora_flan_t5_large_train"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

model.eval()

eval_preds = []

for step, batch in enumerate(tqdm(eval_dataloader)):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    eval_preds.extend \
        (tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True))

# f1 = open('./lora_prediction_flan_t5_base_new.txt', 'w')
f1 = open('./lora_prediction_flan_t5_large_new.txt', 'w')
for pred, true in zip(eval_preds, dataset_test["text_label"]):
    print(pred, true)
    f1.write('True: ' + true + ' Pred: ' + pred)
    f1.write('\n')

f1.close()

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
[codecarbon WARNING @ 01:21:47] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 01:21:47] [setup] RAM Tracking...
[codecarbon INFO @ 01:21:47] [setup] GPU Tracking...
[codecarbon INFO @ 01:21:47] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 01:21:47] [setup] CPU Tracking...
[codecarbon INFO @ 01:21:47] Tracking Intel CPU via RAPL interface
[codecarbon INFO @ 01:21:48] >>> Tracker's metadata:
[codecarbon INFO @ 01:21:48]   Platform system: Linux-5.15.0-139-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 01:21:48]   Python version: 3.10.14
[codecarbon INFO @ 01:21:48]   CodeCarbon version: 2.4.2
[codecarbon INFO @ 01:21:48]   Available RAM : 125.517 GB
[codecarbon INFO @ 01:21:49]   CPU count: 40

Step,Training Loss
500,0.364200
1000,0.079500
1500,0.063800
2000,0.055800
2500,0.051100
3000,0.046900


[codecarbon INFO @ 01:22:18] Energy consumed for RAM : 0.000196 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 01:22:18] Energy consumed for all GPUs : 0.001026 kWh. Total GPU Power : 245.86685876235916 W
[codecarbon INFO @ 01:22:18] Energy consumed for all CPUs : 0.000230 kWh. Total CPU Power : 55.05512140296504 W
[codecarbon INFO @ 01:22:18] 0.001452 kWh of electricity used since the beginning.
[codecarbon INFO @ 01:22:33] Energy consumed for RAM : 0.000392 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 01:22:33] Energy consumed for all GPUs : 0.002110 kWh. Total GPU Power : 260.5575270510689 W
[codecarbon INFO @ 01:22:33] Energy consumed for all CPUs : 0.000459 kWh. Total CPU Power : 55.1665212890605 W
[codecarbon INFO @ 01:22:33] 0.002962 kWh of electricity used since the beginning.
[codecarbon INFO @ 01:22:48] Energy consumed for RAM : 0.000588 kWh. RAM Power : 47.068857192993164 W
[codecarbon INFO @ 01:22:48] Energy consumed for all GPUs : 0.003209 kWh. Total 

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

100%|██████████| 124/124 [00:10<00:00, 11.84it/s]


er Conspiracy — The tweet suggests some deeper conspiracy, and not just that the Big Pharma want to make money (e.g Deeper Conspiracy — The tweet suggests some deeper conspiracy, and not just that the Big Pharma want to make money (e.g., vaccines are being used to track people, COVID is a hoax).Against Big Pharma — The tweet indicates that the Big Pharmaceutical companies are just trying to earn money, or the tweet is against such companies in general because of their history.
Vaccine is ineffective — The tweet expresses concerns that the vaccines are not effective enough and are useless. Vaccine is ineffective — The tweet expresses concerns that the vaccines are not effective enough and are useless.
 specific reason stated in the tweet, or some reason other than the given ones. The No No No      No specific reason stated in the tweet, or some reason other than the given ones.
UnAgainst Big Pharma — The tweet indicates that the Big Pharmaceutical companies are just trying to earn money

In [12]:
import evaluate
from evaluate import load  # Correct import for metrics
metric = evaluate.load("rouge")

Using the latest cached version of the module from /home/sohampoddar/HDD2/hfcache/modules/evaluate_modules/metrics/evaluate-metric--rouge/b01e0accf3bd6dd24839b769a5fda24e14995071570870922c71970b3a6ed886 (last modified on Thu Jun 13 01:49:33 2024) since it couldn't be found locally at evaluate-metric--rouge, or remotely on the Hugging Face Hub.


In [15]:
# text_file_path = './lora_prediction_flan_t5_base.txt'
text_file_path = './lora_prediction_flan_t5_large_new.txt'
with open(text_file_path, 'r') as file:
    lines = file.readlines()
    true_labels = []
    predictions = []
    # split the line by 'Pred:'
    for line in lines:
        if 'True:' in line and 'Pred:' in line:
            true_label = line.split('True: ')[1].split('Pred:')[0].strip()
            prediction = line.split('Pred: ')[1].strip()
            true_labels.append(true_label)
            predictions.append(prediction)


# Calculate the metrics rouge L, rouge 1, and rouge 2
result = metric.compute(predictions=predictions, references=true_labels)
import numpy as np
rouge1_mean = np.mean(result['rouge1']) * 100
rouge2_mean = np.mean(result['rouge2']) * 100
rougeL_mean = np.mean(result['rougeL']) * 100
rougeLsum_mean = np.mean(result['rougeLsum']) * 100

print(f"Rouge1: {rouge1_mean:.2f}")
print(f"Rouge2: {rouge2_mean:.2f}")
print(f"RougeL: {rougeL_mean:.2f}")
print(f"RougeLsum: {rougeLsum_mean:.2f}")


Rouge1: 86.97
Rouge2: 86.04
RougeL: 86.68
RougeLsum: 86.70


In [16]:
from bert_score import score
P, R, F1 = score(predictions,true_labels, lang="en", verbose=True)
# Display BERTScore results
print("BERTScore Precision:", P.mean().item())
print("BERTScore Recall:", R.mean().item())
print("BERTScore F1:", F1.mean().item())

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/16 [00:00<?, ?it/s]

done in 0.67 seconds, 1465.50 sentences/sec
BERTScore Precision: 0.9750670790672302
BERTScore Recall: 0.9725419878959656
BERTScore F1: 0.9734387397766113
